## DistilBERT Fine-Tuning 

In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import Trainer, TrainingArguments

import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import helpers as helpers
import torch.nn as nn

/Users/griffinmichalak/Desktop/code/fall25/nlp/nlp-final-project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/griffinmichalak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# DistilBERT: max_len=128–256, batch_size=16, lr≈2e-5, epochs=2–3, early stopping.

DEPRESSION_PATH = "data/depression_dataset_reddit_cleaned.csv"
TDT_SPLIT = "80/10/10"
BERT_MODEL = 'distilbert-base-uncased'

MAX_LENGTH = 256
BATCH_SIZE = 16
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

In [3]:
data, labels = helpers.get_data()
train_text, dev_text, test_text = helpers.split(data=data, dist=TDT_SPLIT)
train_dep, dev_dep, test_dep = helpers.split(data=labels, dist=TDT_SPLIT)

# convert data, labels into a pandas DataFrame
df = pd.DataFrame({'clean_text': data, 'is_depression': labels})

Completing 80/10/10 split
Completing 80/10/10 split


In [4]:
# initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(BERT_MODEL)
bert = DistilBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# stats from tokenizers
print("Vocab length is : ",len(tokenizer.vocab))
print(tokenizer.decode(2897))

vocab = list(tokenizer.vocab.keys())
print(f"first 10: {vocab[:10]}")
print(f"1280-1290: {vocab[1280:1290]}")
print(f"last 10: {vocab[-10:]}")

print(f"token 2897: {tokenizer.decode(2897)}")
print(f"token 102: {tokenizer.decode(102)}")
print(f"token 103: {tokenizer.decode(103)}")
if 'alex' in tokenizer.vocab:
  print(f"alex id: {tokenizer.vocab['alex']}")
else:
  print("alex not found")

Vocab length is :  30522
network
first 10: ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]']
1280-1290: ['ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ']
last 10: ['##！', '##（', '##）', '##，', '##－', '##．', '##／', '##：', '##？', '##～']
token 2897: network
token 102: [SEP]
token 103: [MASK]
alex id: 4074


In [6]:
# tokenize train, dev, and test
train_encodings = tokenizer(train_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")
dev_encodings = tokenizer(dev_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")

In [ ]:
# custom dataset class
class DepressionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }
        return item
    
    def __len__(self):
        return len(self.labels)

In [8]:
# Convert labels to tensors
train_labels = torch.tensor(train_dep)
dev_labels = torch.tensor(dev_dep)
test_labels = torch.tensor(test_dep)

# Create datasets using the custom class
train_dataset = DepressionDataset(train_encodings, train_labels)
dev_dataset = DepressionDataset(dev_encodings, dev_labels)
test_dataset = DepressionDataset(test_encodings, test_labels)

# Create DataLoaders (for manual training if needed)
dl_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dl_dev = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)
dl_test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
optimizer = AdamW(bert.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_fn = nn.CrossEntropyLoss()

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions),
        'precision': precision_score(labels, predictions),
        'recall': recall_score(labels, predictions)
    }

In [11]:
# begin training
# using hugging face built in trainer https://huggingface.co/blog/davidberenstein1957/fine-tune-modernbert-on-synthetic-data#:~:text=the%20training%20arguments.-,from%20huggingface_hub%20import%20HfFolder,-from%20transformers%20import

training_args = TrainingArguments(
    output_dir="./distilbert-depression-classifier",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=2e-5,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

/Users/griffinmichalak/Desktop/code/fall25/nlp/nlp-final-project/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.092800,0.093319,0.971539,0.971204,0.997312,0.946429
2,0.055100,0.077907,0.981889,0.982005,0.989637,0.974490
3,0.027900,0.081369,0.981889,0.982051,0.987113,0.977041


/Users/griffinmichalak/Desktop/code/fall25/nlp/nlp-final-project/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/griffinmichalak/Desktop/code/fall25/nlp/nlp-final-project/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1161, training_loss=0.07839657939989746, metrics={'train_runtime': 595.8979, 'train_samples_per_second': 31.133, 'train_steps_per_second': 1.948, 'total_flos': 1228767589933056.0, 'train_loss': 0.07839657939989746, 'epoch': 3.0})